In [1]:
#importing libraries

import cv2
import os
import numpy as np
import pandas as pd
import pytesseract
from pytesseract import Output
import matplotlib.pyplot as plt
pytesseract.pytesseract.tesseract_cmd="/home/aman/anaconda3/envs/tallyInvoiceParser.env/bin/tesseract"
os.environ['TESSDATA_PREFIX'] = "/home/aman/anaconda3/envs/tallyInvoiceParser.env/share/tessdata"
import import_ipynb
from imagePreProcessor import *
from TesseractModify import *
from datetime import datetime
import math
import dateparser
import datefinder
from IPython.display import clear_output
import difflib

importing Jupyter notebook from imagePreProcessor.ipynb
importing Jupyter notebook from TesseractModify.ipynb


In [2]:
imagesFolderPath = '/home/aman/Documents/Tally/Git-Document-AI/Document-AI/Data/Images'

In [3]:
def preProcessDataFrame(df, image):
    
    #drop null values
    df.dropna(inplace=True)
    
    
    #rows having text as a stopword removed
    indexesToDrop = []
    stopwords = ["", " ", "  ", "/", ":", "-", ".", ",", "\n", "\t", "\\", "(", ")", "[", "]", "{", "}", "*", "&", "%", "$", "#", "|"]
    for index in df.index:
        if df["text"][index] in stopwords:
            indexesToDrop.append(index)   
    df.drop(indexesToDrop, inplace=True)
    
    #resetting the index
    df.reset_index(inplace=True)
    df.drop(columns=["index"], inplace=True)

    for index in df.index:
        df["text"][index] = df["text"][index].lower()
    
    #creating new columns for centroid information
    df["x"] = df["left"] + df["width"]//2 
    df["y"] = df["top"] + df["height"]//2
    
    #new columns for page width and height for further normalization
    df["PageHeight"] = image.shape[0]
    df["PageWidth"] = image.shape[1]

    df["x"] /= df["PageWidth"]
    df["y"] /= df["PageHeight"]
    
    #adding columns for zonal info (4 Zone)
    df["isTop"] = 0
    df["isBottom"] = 0
    df["isRight"] = 0
    df["isLeft"] = 0

In [4]:
def verifyDate(text):
    
    dates = list(datefinder.find_dates(text, strict=True))
    
    if len(dates)==0:
        return False
    else:
        for date in dates:
            year = date.year
            presentYear = date.today().year
            if abs(year-presentYear)<=2:
                print(dates)
                return True
            else:
                return False

In [5]:
def findDate(dateDF):
    dates = []
    
    for index in dateDF.index:
        text = dateDF["text"][index]
        if verifyDate(text):
            dates.append(text)
    return dates

In [6]:
def extractDateDataFrame(df):
    dateDF = df.copy()
    for index in dateDF.index:
        text = dateDF["text"][index]
        dateDF["text"][index] = text.lower()
    dates = findDate(dateDF)
    return dateDF, dates
    
        

In [7]:
def returnDist(x1, y1, x2, y2):
    p = [x1, y1]
    q = [x2, y2]
    return math.dist(p, q)

In [8]:
def compare(word, neighbours):
    if word in neighbours:
        return word
    else:
        matches = difflib.get_close_matches(word, neighbours, n=1, cutoff=0.8)
        if len(matches)==0:
            return None
        else:
            return matches[0]
            

In [26]:
def allocateNeighbours(df, neighbourFields, neighbourFieldsX, neighbourFieldsY):
    for index in df.index:
        x1 = df["x"][index]
        y1 = df["y"][index]
        block = df["block_num"][index]
        neighbours = {}
        # pw = df["PageWidth"][index]
        # ph = df["PageHeight"][index]
        # pd = np.sqrt((pw**2) + (ph**2))
        distFact = 0.3
        for index2 in df.index:
            x2 = df["x"][index2]
            y2 = df["y"][index2]
            block2 = df["block_num"][index2]
            dist = returnDist(x1, y1, x2, y2)
            if x2<=x1 and y2<=y1:
                if dist<=distFact:
                    if df["text"][index2].lower() not in neighbours:
                        neighbours[df["text"][index2].lower()]=[x2, y2]
                    else:
                        if dist<=returnDist(x1, y1, neighbours[df["text"][index2].lower()][0], neighbours[df["text"][index2].lower()][1]):
                            neighbours[df["text"][index2].lower()]=[x2, y2]
                elif abs(block2-block)<=1:
                    if df["text"][index2].lower() not in neighbours:
                        neighbours[df["text"][index2].lower()]=[x2, y2]
                    else:
                        if dist<=returnDist(x1, y1, neighbours[df["text"][index2].lower()][0], neighbours[df["text"][index2].lower()][1]):
                            neighbours[df["text"][index2].lower()]=[x2, y2]
                    
        for i in range(len(neighbourFields)):
            n = neighbourFields[i]
            nx = neighbourFieldsX[i]
            ny = neighbourFieldsY[i]
            if compare(n, neighbours.keys())==None:
                pass
            else:
                if n=="due date":
                    df[n][index]=-1
                else:
                    df[n][index]=1
                coord = neighbours[compare(n, neighbours.keys())]
                df[nx][index]=coord[0]
                df[ny][index]=coord[1]

        df["NX"] = 0
        df["NY"] = 0

        for index in df.index:
            X = []
            Y = []
            for i in range(len(neighbourFields)):
                n = df[neighbourFields[i]][index]
                nx = df[neighbourFieldsX[i]][index]
                ny = df[neighbourFieldsY[i]][index]
                X.append(n*nx)
                Y.append(n*ny)
            df["NX"][index] = sum(X)
            df["NY"][index] = sum(Y)







In [27]:
def addNeighbours(df):
    neighbourFields = ["invoice", "invoice date", "invoice dt", "due date", "date", "dated"]
    
    neighbourFieldsX = ["invoice x", "invoice date x", "invoice dt x", "due date x", "date x", "dated x"]

    neighbourFieldsY = ["invoice y", "invoice date y", "invoice dt y", "due date y", "date y", "dated y"]

    for col in neighbourFields:
        df[col]=0
    for col in neighbourFieldsX:
        df[col]=0
    for col in neighbourFieldsY:
        df[col]=0
    allocateNeighbours(df, neighbourFields, neighbourFieldsX, neighbourFieldsY)
    

In [28]:
def findDateDF(dates, dateDF):
    indexes = []
    for date in dates:
        i = dateDF[dateDF["text"]==date].index
        for index in i:
            indexes.append(index)
    return indexes

In [29]:
def dropIndexes(indexes, df):
    indexesToDrop=[]
    for index in df.index:
        if index not in indexes:
            indexesToDrop.append(index)
    df.drop(indexesToDrop, inplace=True)

In [30]:
def create(imagePath):
    image = cv2.imread(imagePath)
    preProcessedImage = preProcessImage(imagePath)
    data = pytesseract.image_to_data(preProcessedImage, output_type=Output.DATAFRAME)
    data.dropna(inplace=True)
    data.reset_index(inplace=True)
    data.drop(columns=["index"], inplace=True)
    processData(data)
    df = pd.DataFrame(data)
    preProcessDataFrame(df, preProcessedImage)
    dateDF, dates = extractDateDataFrame(df)
    addNeighbours(df)
    indexes = findDateDF(dates, dateDF)
    resultDF = df.copy()
    dropIndexes(indexes, resultDF)
    return resultDF
    
    

In [31]:
def batchCreate(folderPath):
    os.chdir(folderPath)
    images = os.listdir()
    dfList = []
    for imagePath in images:
        #print(imagePath)
        df = create(imagePath)
        df["imageName"] = imagePath
        df["output"] = 0
        dfList.append(df)
    return dfList

In [32]:
os.chdir(imagesFolderPath)

In [33]:
imagePath = "28.jpeg"

In [34]:
df = create(imagePath)
df

<string>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/tmp/ipykernel_6320/909959028.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["text"][index] = df["text"][index].lower()
/tmp/ipykernel_6320/3296310189.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dateDF["text"][index] = text.lower()
/tmp/ipykernel_6320/235416507.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See 

[datetime.datetime(2023, 2, 1, 0, 0)]
[datetime.datetime(2023, 2, 1, 0, 0)]
[datetime.datetime(2022, 1, 12, 0, 0)]
[datetime.datetime(2022, 12, 31, 0, 0)]
[datetime.datetime(2022, 1, 12, 0, 0)]
[datetime.datetime(2022, 12, 31, 0, 0)]
[datetime.datetime(2023, 2, 1, 0, 0)]
[datetime.datetime(2023, 2, 1, 0, 0)]
[datetime.datetime(2022, 1, 12, 0, 0), datetime.datetime(2022, 12, 31, 0, 0)]
[datetime.datetime(2022, 1, 12, 0, 0), datetime.datetime(2022, 12, 31, 0, 0)]


/tmp/ipykernel_6320/235416507.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[n][index]=1
/tmp/ipykernel_6320/235416507.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[nx][index]=coord[0]
/tmp/ipykernel_6320/235416507.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[ny][index]=coord[1]
/tmp/ipykernel_6320/235416507.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame



,block_num,line_num,left,top,width,height,conf,text,nextDist,x,...,date x,dated x,invoice y,invoice date y,invoice dt y,due date y,date y,dated y,NX,NY
48,4,2,1908,968,211,33,94.371849,02/01/2023.,-1856,0.830980,...,0.739216,0.739216,0.286061,0.252424,0.252424,0.000000,0.303333,0.303333,2.800784,1.397576
61,4,3,1908,1025,205,39,96.413490,02/01/2023,-1849,0.828627,...,0.739608,0.739608,0.286061,0.252424,0.252424,0.322424,0.322424,0.322424,2.065882,1.113333
132,8,5,642,2165,213,35,96.445099,01-12-2022,12,0.335294,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
134,8,5,915,2161,211,36,96.801933,31-12-2022,-844,0.441569,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
159,8,10,644,2425,211,37,96.134308,01-12-2022,13,0.335294,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
161,8,10,915,2424,211,38,96.615639,31-12-2022,-866,0.441569,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
313,4,0,1908,968,211,33,94.371849,02/01/2023.,-1856,0.830980,...,0.739216,0.739216,0.286061,0.252424,0.252424,0.000000,0.303333,0.303333,2.800784,1.397576
325,4,0,1908,1025,205,39,96.413490,02/01/2023,-1849,0.828627,...,0.739608,0.739608,0.286061,0.252424,0.252424,0.322424,0.322424,0.322424,2.065882,1.113333
382,8,0,642,2165,458,36,96.405688,01-12-2022 to 31-12-2022,-844,0.431373,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
399,8,0,644,2425,455,38,96.534648,01-12-2022 to 31-12-2022,-866,0.430980,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [35]:
for col in ["invoice", "invoice date", "invoice dt", "due date", "date", "dated"]:
    print(col, df[col][61])
print("\n")
for col in ["invoice x", "invoice date x", "invoice dt x", "due date x", "date x", "dated x"]:
    print(col, df[col][61])
print("\n")
for col in ["invoice y", "invoice date y", "invoice dt y", "due date y", "date y", "dated y"]:
    print(col, df[col][61])


invoice 1
invoice date 1
invoice dt 1
due date -1
date 1
dated 1


invoice x 0.7027450980392157
invoice date x 0.30980392156862746
invoice dt x 0.30980392156862746
due date x 0.735686274509804
date x 0.7396078431372549
dated x 0.7396078431372549


invoice y 0.28606060606060607
invoice date y 0.25242424242424244
invoice dt y 0.25242424242424244
due date y 0.32242424242424245
date y 0.32242424242424245
dated y 0.32242424242424245


In [36]:
pwd

'/home/aman/Documents/Tally/Git-Document-AI/Document-AI/Data/Images'

In [38]:
testingPath = '/home/aman/Documents/Tally/Git-Document-AI/Document-AI/Data/TestingImages/'

In [39]:
dfList = batchCreate(imagesFolderPath)

<string>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/tmp/ipykernel_6320/909959028.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["text"][index] = df["text"][index].lower()
/tmp/ipykernel_6320/3296310189.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dateDF["text"][index] = text.lower()
/tmp/ipykernel_6320/235416507.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See 

[datetime.datetime(2022, 5, 12, 0, 0)]
[datetime.datetime(2022, 5, 12, 0, 0)]


/tmp/ipykernel_6320/235416507.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[n][index]=1
/tmp/ipykernel_6320/235416507.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[nx][index]=coord[0]
/tmp/ipykernel_6320/235416507.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[ny][index]=coord[1]
<string>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the d

[datetime.datetime(2022, 5, 12, 0, 0)]
[datetime.datetime(2022, 5, 12, 0, 0)]
[datetime.datetime(2022, 5, 12, 0, 0)]
[datetime.datetime(2022, 5, 12, 0, 0)]
[datetime.datetime(2022, 5, 12, 0, 0)]
[datetime.datetime(2022, 5, 12, 0, 0)]


/tmp/ipykernel_6320/235416507.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["NX"][index] = sum(X)
/tmp/ipykernel_6320/235416507.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["NY"][index] = sum(Y)
/tmp/ipykernel_6320/235416507.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[n][index]=1
/tmp/ipykernel_6320/235416507.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

...

[datetime.datetime(2022, 12, 24, 0, 0)]
[datetime.datetime(2022, 12, 27, 0, 0)]
[datetime.datetime(2022, 12, 24, 0, 0)]
[datetime.datetime(2022, 12, 27, 0, 0)]


<string>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/tmp/ipykernel_6320/909959028.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["text"][index] = df["text"][index].lower()
/tmp/ipykernel_6320/3296310189.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dateDF["text"][index] = text.lower()
/tmp/ipykernel_6320/235416507.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See 

[datetime.datetime(2023, 2, 14, 0, 0)]
[datetime.datetime(2023, 2, 14, 0, 0)]


/tmp/ipykernel_6320/235416507.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[n][index]=1
/tmp/ipykernel_6320/235416507.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[nx][index]=coord[0]
/tmp/ipykernel_6320/235416507.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[ny][index]=coord[1]
<string>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the d

[datetime.datetime(2022, 9, 28, 0, 0)]
[datetime.datetime(2022, 9, 28, 0, 0)]


In [40]:
len(dfList)

110

In [41]:
resultDF = pd.concat(dfList)

In [42]:
resultDF.describe()

,block_num,line_num,left,top,width,height,conf,nextDist,x,y,...,dated x,invoice y,invoice date y,invoice dt y,due date y,date y,dated y,NX,NY,output
count,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,...,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.0
mean,9.648829,1.632107,1235.505017,1136.431438,190.010033,32.411371,82.248156,-684.695652,0.553672,0.340528,...,0.305510,0.096639,0.057631,0.093107,0.010971,0.172659,0.169034,1.165773,0.578099,0.0
std,8.896521,3.393406,752.495037,738.282392,75.580371,7.629948,20.776512,954.989468,0.283834,0.211995,...,0.315085,0.146878,0.114787,0.145686,0.050114,0.175338,0.173367,1.253832,0.538632,0.0
min,1.000000,0.000000,75.000000,67.000000,98.000000,16.000000,0.000000,-3189.000000,0.059795,0.019529,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
25%,4.000000,0.000000,556.000000,598.500000,154.000000,28.000000,74.454445,-1627.000000,0.305414,0.179615,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
50%,6.000000,1.000000,1367.000000,915.000000,178.000000,32.000000,90.819194,-401.000000,0.606452,0.271742,...,0.149133,0.000000,0.000000,0.000000,0.000000,0.147546,0.147546,0.703345,0.596364,0.0
75%,12.000000,2.000000,1908.000000,1515.500000,203.000000,37.000000,96.216804,118.000000,0.828627,0.450827,...,0.630588,0.167950,0.073696,0.168000,0.000000,0.268037,0.266291,2.017749,0.838182,0.0
max,46.000000,26.000000,3130.000000,3395.000000,821.000000,71.000000,96.945030,2035.000000,0.941556,0.978286,...,0.868871,0.904848,0.853763,0.904848,0.380844,0.790479,0.790479,4.311165,2.623136,0.0


In [43]:
len(resultDF["imageName"].unique())

72

In [44]:
resultDF.reset_index(inplace=True)

In [45]:
resultDF.drop(columns=["index", "nextDist", "block_num", "line_num", "left", "top", "width", "height", "conf"], inplace=True)

In [46]:
resultDF.shape

(299, 31)

In [47]:
resultDF

,text,x,y,PageHeight,PageWidth,isTop,isBottom,isRight,isLeft,invoice,...,invoice y,invoice date y,invoice dt y,due date y,date y,dated y,NX,NY,imageName,output
0,dot::05/12/2022,0.325149,0.336000,3500,2525,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,20.jpeg,0
1,dot::05/12/2022,0.325149,0.336000,3500,2525,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,20.jpeg,0
2,05/12/2022,0.856471,0.170909,3300,2550,0,0,0,0,1,...,0.152121,0.000000,0.152121,0.0,0.152121,0.152121,2.870588,0.608485,23.jpeg,0
3,05/12/2022,0.858431,0.215152,3300,2550,0,0,0,0,1,...,0.152121,0.000000,0.152121,0.0,0.196364,0.196364,2.870588,0.696970,23.jpeg,0
4,05/12/2022,0.785882,0.419394,3300,2550,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.0,0.419091,0.419091,1.389804,0.838182,23.jpeg,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,27-dec-22,0.878177,0.472064,3508,2479,0,0,0,0,1,...,0.454105,0.000000,0.454105,0.0,0.000000,0.000000,0.864058,0.908210,86.jpeg,0
295,14-feb-23,0.739216,0.077576,3300,2550,0,0,0,0,1,...,0.068485,0.033939,0.033939,0.0,0.068485,0.068485,2.967451,0.273333,46.jpeg,0
296,14-feb-23,0.739216,0.077576,3300,2550,0,0,0,0,1,...,0.068485,0.033939,0.033939,0.0,0.068485,0.068485,2.967451,0.273333,46.jpeg,0
297,28-sep-22,0.762404,0.071266,3508,2479,0,0,0,0,1,...,0.040479,0.040479,0.040479,0.0,0.052166,0.052166,3.010488,0.225770,62.jpeg,0


In [48]:
resultDF.describe()

,x,y,PageHeight,PageWidth,isTop,isBottom,isRight,isLeft,invoice,invoice date,...,dated x,invoice y,invoice date y,invoice dt y,due date y,date y,dated y,NX,NY,output
count,299.000000,299.000000,299.000000,299.000000,299.0,299.0,299.0,299.0,299.000000,299.000000,...,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.0
mean,0.553672,0.340528,3459.036789,2572.792642,0.0,0.0,0.0,0.0,0.474916,0.317726,...,0.305510,0.096639,0.057631,0.093107,0.010971,0.172659,0.169034,1.165773,0.578099,0.0
std,0.283834,0.211995,464.527813,322.096359,0.0,0.0,0.0,0.0,0.500208,0.466373,...,0.315085,0.146878,0.114787,0.145686,0.050114,0.175338,0.173367,1.253832,0.538632,0.0
min,0.059795,0.019529,1566.000000,1134.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
25%,0.305414,0.179615,3300.000000,2479.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
50%,0.606452,0.271742,3484.000000,2481.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.149133,0.000000,0.000000,0.000000,0.000000,0.147546,0.147546,0.703345,0.596364,0.0
75%,0.828627,0.450827,3508.000000,2550.000000,0.0,0.0,0.0,0.0,1.000000,1.000000,...,0.630588,0.167950,0.073696,0.168000,0.000000,0.268037,0.266291,2.017749,0.838182,0.0
max,0.941556,0.978286,5313.000000,3542.000000,0.0,0.0,0.0,0.0,1.000000,1.000000,...,0.868871,0.904848,0.853763,0.904848,0.380844,0.790479,0.790479,4.311165,2.623136,0.0


In [49]:
# resultDF["contextPresent"]=0

In [50]:
# contexts = ["invoice date", "invoice", "invoice dt", "dated", "date", "bill dt", "receipt date", "receipt dt", "bill", "receipt"]

In [51]:
# for index in resultDF.index:
#     cont = []
#     for col in contexts:
#         cont.append(resultDF[col][index])
#     if max(cont)>0:
#         resultDF["contextPresent"][index]=1

In [52]:
# resultDF.describe()

In [53]:
# resultDF["contextDist"] = 0

In [54]:
# for index in resultDF.index:
#     if resultDF["contextPresent"][index]==1:
#         cont = []
#         for col in contexts:
#             if resultDF[col][index]>0:
#                 cont.append(resultDF[col][index])
#         dist = min(cont)
#         resultDF["contextDist"][index] = dist


In [55]:
# resultDF.drop(columns=["output"], inplace=True)

In [56]:
# resultDF["output"] = 0

In [57]:
resultDF

,text,x,y,PageHeight,PageWidth,isTop,isBottom,isRight,isLeft,invoice,...,invoice y,invoice date y,invoice dt y,due date y,date y,dated y,NX,NY,imageName,output
0,dot::05/12/2022,0.325149,0.336000,3500,2525,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,20.jpeg,0
1,dot::05/12/2022,0.325149,0.336000,3500,2525,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,20.jpeg,0
2,05/12/2022,0.856471,0.170909,3300,2550,0,0,0,0,1,...,0.152121,0.000000,0.152121,0.0,0.152121,0.152121,2.870588,0.608485,23.jpeg,0
3,05/12/2022,0.858431,0.215152,3300,2550,0,0,0,0,1,...,0.152121,0.000000,0.152121,0.0,0.196364,0.196364,2.870588,0.696970,23.jpeg,0
4,05/12/2022,0.785882,0.419394,3300,2550,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.0,0.419091,0.419091,1.389804,0.838182,23.jpeg,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,27-dec-22,0.878177,0.472064,3508,2479,0,0,0,0,1,...,0.454105,0.000000,0.454105,0.0,0.000000,0.000000,0.864058,0.908210,86.jpeg,0
295,14-feb-23,0.739216,0.077576,3300,2550,0,0,0,0,1,...,0.068485,0.033939,0.033939,0.0,0.068485,0.068485,2.967451,0.273333,46.jpeg,0
296,14-feb-23,0.739216,0.077576,3300,2550,0,0,0,0,1,...,0.068485,0.033939,0.033939,0.0,0.068485,0.068485,2.967451,0.273333,46.jpeg,0
297,28-sep-22,0.762404,0.071266,3508,2479,0,0,0,0,1,...,0.040479,0.040479,0.040479,0.0,0.052166,0.052166,3.010488,0.225770,62.jpeg,0


In [58]:
for index in resultDF.index:
    name = resultDF["imageName"][index].split(".")[0]
    resultDF["imageName"][index]=int(name)

/tmp/ipykernel_6320/2442472026.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resultDF["imageName"][index]=int(name)


In [59]:
resultDF.sort_values("imageName", inplace=True)
resultDF

,text,x,y,PageHeight,PageWidth,isTop,isBottom,isRight,isLeft,invoice,...,invoice y,invoice date y,invoice dt y,due date y,date y,dated y,NX,NY,imageName,output
72,21-07-2022,0.772892,0.081050,3504,2479,0,0,0,0,1,...,0.038813,0.038813,0.038813,0.0,0.067922,0.067922,2.963292,0.252283,1,0
73,21-07-2022,0.772892,0.081050,3504,2479,0,0,0,0,1,...,0.038813,0.038813,0.038813,0.0,0.067922,0.067922,2.963292,0.252283,1,0
206,1-oct-22. to 31,0.751109,0.561358,3504,2479,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.0,0.493721,0.493721,1.342477,0.987443,2,0
204,1-oct-22.,0.719645,0.561358,3504,2479,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.0,0.493721,0.493721,1.342477,0.987443,2,0
203,22-sep-22,0.217426,0.148687,3504,2479,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.0,0.147546,0.147546,0.224284,0.295091,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151,01-feb-2023,0.909020,0.375455,3300,2550,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,97,0
270,21-08-2022,0.910448,0.160775,3508,2479,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.0,0.160775,0.160775,1.681323,0.321551,98,0
88,07-feb-2023,0.740741,0.340358,1566,1134,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,100,0
143,01/10/2022,0.770968,0.142002,3507,2480,0,0,0,0,1,...,0.127745,0.127745,0.127745,0.0,0.141146,0.141146,3.122581,0.665526,107,0


In [60]:
# for index in resultDF.index:
#     name = str(resultDF["imageName"][index]) + ".jpeg"
#     resultDF["imageName"][index]=name
# resultDF

In [61]:
os.chdir('/home/aman/Documents/Tally/Git-Document-AI/Document-AI/InvoiceDateModel-v2/Dataset/')

In [62]:
pwd

'/home/aman/Documents/Tally/Git-Document-AI/Document-AI/InvoiceDateModel-v2/Dataset'

In [63]:
resultDF.to_csv("tesseract6.csv")